In [3]:
import pandas as pd

In [4]:
from itertools import product
from math import sqrt
import numpy as np


class Heuristic():

    def __init__(self, size):
        self.size = size
        self.len = size * size

    def get_distance_one_num(self, row, col):
        pass

    def get_distance(self):
        d_map = {((row * self.size) + col) : self.get_distance_one_num(row, col)
                 for row, col in product(range(self.size), range(self.size))}
        return d_map

    def get_h(self, puzzle, solved):
        all_h = [self.dist_map[num][solved.index(puzzle[num])]
                 for num in range(self.len)]
#         print(all_h) #test
        return sum(all_h)


class Euclidian(Heuristic):

    def __init__(self, size):
        super().__init__(size)
        self.dist_map = self.get_distance()
        
    def get_distance_one_num(self, row, col):
        return [sqrt((t_row - row)**2 + (t_col - col)**2)
                    for t_row, t_col in product(range(self.size), range(self.size))]

    
class Manhattan(Heuristic):

    def __init__(self, size):
        super().__init__(size)
        self.dist_map = self.get_distance()
        
    def get_distance_one_num(self, row, col):
        return [abs(t_row - row) + abs(t_col - col)
                    for t_row, t_col in product(range(self.size), range(self.size))]

    
class Hamming(Heuristic):

    def __init__(self, size):
        super().__init__(size)
        
    def get_h(self, puzzle, solved):
        return np.count_nonzero(np.array(puzzle) - np.array(solved))

In [5]:
puzzle = [1, 2, 3, 0, 6, 4, 8, 7, 5]
target = [1, 2, 3, 8, 0, 4, 7, 6, 5]
psize = 3

In [6]:
heuristic = Manhattan(psize)

In [7]:
# type(states.iloc[0])
# current = states.iloc[0]

In [8]:
# current.name

In [9]:
def move_up(puzzle, hole, psize):
    if hole + 1 > psize:
        new = puzzle[:]
        new[hole], new[hole - psize] = new[hole - psize], new[hole]
        return new
    
def move_down(puzzle, hole, psize):
    if hole + 1 + psize <= psize * psize:
        new = puzzle[:]
        new[hole], new[hole + psize] = new[hole + psize], new[hole]
        return new
    
def move_left(puzzle, hole, psize):
    if (hole) % psize:
        new = puzzle[:]
        new[hole], new[hole - 1] = new[hole - 1], new[hole]
        return new
    
def move_right(puzzle, hole, psize):
    if (hole + 1) % psize:
        new = puzzle[:]
        new[hole], new[hole + 1] = new[hole + 1], new[hole]
        return new
    
moves = [move_up, move_down, move_left, move_right]

In [10]:
# move_left([15, 10, 14, 5, 13, 3, 0, 7, 8, 11, 1, 9, 6, 2, 12, 4],15, 4) 
move_right([15, 10, 14, 5, 13, 3, 0, 7, 8, 11, 1, 9, 6, 2, 12, 4],7, 4) 

In [54]:
# from State import State
import pandas as pd
# from moves import moves

class Solver:

    def __init__(self, puzzle, psize, target, heuristic, f_calculation = (lambda g, h: g + h)):
        self.initial = puzzle
        self.solved_puzzle_hash = str(target)
        self.solved_puzzle = target
        self.psize = psize
        self.states = pd.DataFrame({'puzzle': [puzzle], 'parent': None, 'g': 0,
                       'f': heuristic.get_h(puzzle, target), 'is_open_list': 1},
                        index=[(str(puzzle))])
        self.heuristic = heuristic
        self.f_calculation = f_calculation

    def a_star(self):
        while self.states['is_open_list'].sum():
#             print(self.states) # tst
            current = self.states[self.states.is_open_list == 1].iloc[0]
            if current.name == self.solved_puzzle_hash:
                return self.get_result(current)
            else:
                self.states.loc[current.name, 'is_open_list'] = 0
                self.expand(current)
                self.states = self.states.sort_values('f')


    def expand(self, current):
        hole_index = current.puzzle.index(0)
        expanded = [foo(current.puzzle, hole_index, self.psize) for foo in moves]
        g = current.g + 1
        for i in expanded:
            if i:
                f = self.f_calculation(g, self.heuristic.get_h(i, self.solved_puzzle))
                if str(i) in self.states.index:
                    if f < self.states.loc[str(i)]['f']:
                        self.states.loc[str(i)] = [i, current.name, g, f, 1]
                else:
                    self.states.loc[str(i)] = [i, current.name, g, f, 1]
                if str(i) == self.solved_puzzle_hash:
                    return ;

    def print_states(self, idx, size):
        if self.states.loc[idx, 'parent']:
            self.print_states(self.states.loc[idx, 'parent'], size)
        print('step ', self.states.loc[idx, 'g'])
        chunks = (self.states.loc[idx, 'puzzle'][i : i + size]
               for i in range(0, size*size, size))
        for ch in chunks:
            print(ch)

    def get_result(self, current):
#         print(type(current.name))
#         print(self.states.loc[current.name, 'parent'])
        self.print_states(current.name, self.psize)
#         print('finita')
#         return self.states

In [55]:
solv = Solver(puzzle, 3, target, heuristic)

In [56]:
solv.a_star()

step  0
[1, 2, 3]
[0, 6, 4]
[8, 7, 5]
step  1
[1, 2, 3]
[8, 6, 4]
[0, 7, 5]
step  2
[1, 2, 3]
[8, 6, 4]
[7, 0, 5]
step  3
[1, 2, 3]
[8, 0, 4]
[7, 6, 5]


In [87]:
[1, 2, 3, 8, 0, 4, 7, 6, 5]
trr = move([15, 10, 5, 0, 13, 3, 14, 7, 8, 11, 1, 9, 6, 2, 12, 4], 4)     
trr
# for i in range(4):
#     print(i)

[None,
 [15, 10, 5, 7, 13, 3, 14, 0, 8, 11, 1, 9, 6, 2, 12, 4],
 [15, 10, 0, 5, 13, 3, 14, 7, 8, 11, 1, 9, 6, 2, 12, 4],
 None]

In [44]:
trr[0][0] = 'fdksl'

In [46]:
trr[0].index(0)

6

In [100]:
states.loc[str([15, 10, 0, 5, 13, 3, 14, 7, 8, 11, 1, 9, 6, 2, 12, 4])] = [[15, 10, 0, 5, 13, 3, 14, 7, 8, 11, 1, 9, 6, 2, 12, 4], ['some'], 1, 89, 1]

In [101]:
states

,puzzle,parent,g,f,is_open_list
"[15, 10, 14, 5, 13, 3, 0, 7, 8, 11, 1, 9, 6, 2, 12, 4]","[15, 10, 14, 5, 13, 3, 0, 7, 8, 11, 1, 9, 6, 2...",None,0,42,1
"[15, 10, 0, 5, 13, 3, 14, 7, 8, 11, 1, 9, 6, 2, 12, 4]","[15, 10, 0, 5, 13, 3, 14, 7, 8, 11, 1, 9, 6, 2...",[some],1,89,1


In [95]:
str([15, 10, 0, 5, 13, 3, 14, 7, 8, 11, 1, 9, 6, 2, 12, 4]) in states.index

True

In [102]:
states.loc[str([15, 10, 0, 5, 13, 3, 14, 7, 8, 11, 1, 9, 6, 2, 12, 4])]['']

89